In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
from tqdm import tqdm
import torch.nn.functional as F
from gensim.models import Word2Vec
from sklearn import metrics
from sklearn.metrics import classification_report
import itertools
import random

In [10]:
import torch
import torch.nn as nn

In [11]:
device = 'cuda'

# 284 915


In [12]:
pos_weight = 915/284

In [13]:
class BertModel(nn.Module):
    def __init__(self, voc_size:int=30000, seq_len: int=512, d_model: int=384, d_ff:int=3072, pad_idx: int=1,
                num_encoder: int=12, num_heads: int=12, n_layers=6, dropout: float=0.1):
        super(BertModel, self).__init__()
        self.pad_idx = pad_idx
        self.layers = nn.ModuleList([nn.TransformerEncoderLayer(d_model=d_model, nhead=8, batch_first=True) for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ1 = nn.Tanh()
        self.classifier = nn.Linear(d_model, 1)
        
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        output = input
        for layer in self.layers:
            output = layer(output)
        #output = output.mean(dim=1, keepdim=True)
        output = output[:,-1]
        h_pooled = self.activ1(self.fc(output)) # [batch_size, d_model]
        logits_clsf = self.classifier(h_pooled) # [batch_size, 2]
        return logits_clsf # [B, S, D_model]

In [14]:
def log_metrics(preds, labels):
    preds = torch.stack(preds)
    preds = preds.cpu().detach().numpy()
    labels = torch.stack(labels)
    labels = labels.cpu().detach().numpy()
    
    
    ## Method 2 using ravel()
    fpr_micro, tpr_micro, _ = metrics.roc_curve(labels.ravel(), preds.ravel())
    auc_micro = metrics.auc(fpr_micro, tpr_micro)
    n=10
    preds = [round(i) for i in preds.ravel()]
    acc = metrics.accuracy_score(labels.ravel(), preds)
    f1 = metrics.f1_score(labels.ravel(), preds)
    preds = [preds[i*n:(i+1)*n] for i in range(len(preds)//6)]
    #target_names = list(genre_list.keys())
    #print(classification_report(labels,preds, target_names=target_names))
    return {"auc_micro": auc_micro, "acc" : acc, "f1" : f1, 'pred' : preds}

In [52]:
def train_one_epoch(dataloader, model, optimizer, device, loss_fn):
    model.train()
    tk0 = tqdm(dataloader, total=len(dataloader))
    total_loss = 0.0
    
    for source_tensor, label in tk0:
        optimizer.zero_grad()
        output = model(source_tensor.to(device))
        loss = loss_fn(output.view(-1, output.size(-1)), torch.FloatTensor([label]).to(device))
        
        total_loss += loss
        loss.backward()
        optimizer.step()
        
    avg_train_loss = total_loss / len(dataloader) 
    print(" Average training loss: {0:.2f}".format(avg_train_loss))  

def test_one_epoch(testset, model, loss_fn):
    model.eval()
    tk0 = tqdm(testset, total=len(testset))
    targets = []
    outputs = []
    total_loss = 0.0
    with torch.no_grad():
        for source_tensor, label in tk0:
            label = torch.FloatTensor([label])
            output = model(source_tensor.to(device))
            loss = loss_fn(output.view(-1, output.size(-1)), label.to(device))
            total_loss += loss
            targets.extend(label)
            outputs.extend(torch.sigmoid(output[0]).to('cpu'))
    avg_valid_loss = total_loss / len(testset) 
    score = log_metrics(outputs, targets)
    print(" Average valid loss: {0:.2f}".format(avg_valid_loss))  
    print('AUC_SCORE: ', score["auc_micro"], " acc: ", score["acc"], "f1: ", score["f1"])
    return score, avg_valid_loss
    
def fit(model, train_dataloader, valid_dataloader=None,test_dataloader=None, EPOCHS=3, lr=0.0000005):
    loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weight)) #ignore padding
    optimizer = torch.optim.AdamW(model.parameters(),lr=lr)
    for i in range(EPOCHS):
        print(f"EPOCHS:{i+1}")
        print('TRAIN')
        train_one_epoch(train_dataloader, model, optimizer, device, loss_fn)
        auc, loss = test_one_epoch(valid_dataloader,model, loss_fn)
        if i == 0:
            torch.save(model,'rnn_best.model')
            best_loss = loss
        if  loss<best_loss:
            best_loss = loss
            print(i,' EPOCH BEST MODEL!')
            torch.save(model,'rnn_best.model')
        print('Test')
        auc, loss = test_one_epoch(test_dataloader,model, loss_fn)
    best_model = torch.load('rnn_best.model')
    auc, loss  = test_one_epoch(test_dataloader,best_model, loss_fn)
    print('BEST MODEL')
    print('AUC_SCORE: ', auc["auc_micro"], " acc: ", auc["acc"], "f1: ", auc["f1"])
    return auc

            best_f1 = auc['f1']
        if  auc['f1']>best_f1:
            best_f1 = auc['f1']

In [27]:
import pickle
with open('movie_split_surprise.pickle','rb') as fw:
    movie_split = pickle.load(fw)

In [97]:
path1 = 'word2vec_model_edge/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
end = torch.ones(1,1,400)
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'surprise ending'==keyword:
                    cult = 1
                    
                    
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,400)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])

In [99]:
bert = BertModel(d_model=400).to('cuda')
score = fit(bert, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|█████████████████████████████████████████| 963/963 [00:12<00:00, 79.87it/s]


 Average training loss: 1.07


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 461.68it/s]


 Average valid loss: 1.05
AUC_SCORE:  0.562111801242236  acc:  0.3333333333333333 f1:  0.375
Test


...

 Average valid loss: 1.04
AUC_SCORE:  0.6929347826086957  acc:  0.6416666666666667 f1:  0.48192771084337355
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 852.89it/s]


 Average valid loss: 0.89
AUC_SCORE:  0.7691154422788605  acc:  0.71900826446281 f1:  0.5


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 850.28it/s]

 Average valid loss: 0.91
AUC_SCORE:  0.7616191904047976  acc:  0.7024793388429752 f1:  0.5384615384615385
BEST MODEL
AUC_SCORE:  0.7616191904047976  acc:  0.7024793388429752 f1:  0.5384615384615385
{'auc_micro': 0.7616191904047976, 'acc': 0.7024793388429752, 'f1': 0.5384615384615385, 'pred': [[0, 0, 1, 1, 0, 1, 1, 0, 0, 0], [0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 1, 0, 0], [1, 1, 0, 0, 1, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 1, 1, 0], [1, 1, 0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 1, 0, 0, 0, 1, 1, 0, 0], [0, 1, 1, 1, 1, 1, 0, 1, 0, 0], [1, 0, 0, 0, 1, 1, 0, 1, 0, 1], [0, 0, 0, 0, 1, 0, 1, 1, 1, 0], [1, 1, 0, 0, 0, 0, 1, 1, 0, 0], [0], [], [], [], [], [], [], []]}


In [104]:
random.randint(0,100)

19

In [69]:
import random

# Doc2vce

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# SUM

In [76]:
import pickle
with open('emb_sum.pickle','rb') as fw:
    movie_emb = pickle.load(fw)

In [81]:
path1 = 'word2vec_model_char/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
end = torch.ones(1,1,384)

for filename in filenames:
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'surprise ending'==keyword:
                    cult = 1
    if movie_name not in movie_emb.keys():
        continue
        
    embs = torch.zeros(1,len(movie_emb[movie_name])+1,384)
    for i,emb in enumerate(movie_emb[movie_name]):
        emb = torch.from_numpy(np.nan_to_num(emb))
        embs[0][i] = emb
        
    embs[0][-1] = end
        
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
        #valid_data.append([embs, [cult]])

In [82]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

953
120
119


In [83]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 953/953 [00:08<00:00, 112.10it/s]


 Average training loss: 1.07


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 828.85it/s]

 Average valid loss: 1.06
AUC_SCORE:  0.49805900621118016  acc:  0.25833333333333336 f1:  0.3776223776223776


...

 Average valid loss: 1.01
AUC_SCORE:  0.6444099378881988  acc:  0.49166666666666664 f1:  0.4077669902912622
Test


100%|████████████████████████████████████████| 119/119 [00:00<00:00, 848.04it/s]


 Average valid loss: 0.99
AUC_SCORE:  0.6703296703296704  acc:  0.6050420168067226 f1:  0.4946236559139785


100%|████████████████████████████████████████| 119/119 [00:00<00:00, 842.97it/s]

 Average valid loss: 0.99
AUC_SCORE:  0.6695447409733124  acc:  0.5798319327731093 f1:  0.47916666666666674
BEST MODEL
AUC_SCORE:  0.6695447409733124  acc:  0.5798319327731093 f1:  0.47916666666666674
{'auc_micro': 0.6695447409733124, 'acc': 0.5798319327731093, 'f1': 0.47916666666666674, 'pred': [[0, 0, 1, 1, 0, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 0, 1, 1, 1, 1], [0, 1, 0, 1, 0, 0, 0, 0, 1, 0], [1, 1, 1, 0, 1, 1, 0, 1, 1, 0], [1, 0, 1, 1, 0, 0, 1, 1, 0, 0], [0, 0, 1, 0, 0, 1, 0, 1, 1, 1], [1, 0, 1, 1, 1, 0, 0, 1, 1, 1], [1, 1, 0, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 0, 0, 1, 0, 0, 1], [1, 0, 1, 1, 0, 1, 1, 0, 1, 0], [0, 0, 0, 0, 0, 1, 1, 1, 0, 1], [1, 1, 1, 0, 1, 0, 1, 1, 0], [], [], [], [], [], [], []]}


# Doc2vec

In [78]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [79]:
path1 = 'doc2vec/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'surprise ending'==keyword:
                    cult = 1
        
    w2v=Doc2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.dv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.dv.index_to_key:
            emb = w2v.dv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [80]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:08<00:00, 113.96it/s]


 Average training loss: 1.07


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 844.44it/s]

 Average valid loss: 1.04
AUC_SCORE:  0.5597826086956522  acc:  0.49166666666666664 f1:  0.3711340206185567


...

 Average valid loss: 1.25
AUC_SCORE:  0.4794254658385093  acc:  0.5333333333333333 f1:  0.3170731707317073
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 883.43it/s]


 Average valid loss: 1.12
AUC_SCORE:  0.5659670164917542  acc:  0.5454545454545454 f1:  0.367816091954023


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 880.47it/s]

 Average valid loss: 1.06
AUC_SCORE:  0.5637181409295353  acc:  0.4793388429752066 f1:  0.37623762376237624
BEST MODEL
AUC_SCORE:  0.5637181409295353  acc:  0.4793388429752066 f1:  0.37623762376237624
{'auc_micro': 0.5637181409295353, 'acc': 0.4793388429752066, 'f1': 0.37623762376237624, 'pred': [[0, 1, 1, 1, 1, 1, 0, 1, 0, 0], [1, 1, 1, 1, 1, 0, 0, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 1, 0, 1], [1, 0, 0, 1, 1, 1, 1, 0, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0, 1, 0], [0, 1, 1, 1, 0, 1, 0, 1, 0, 1], [0, 0, 1, 0, 0, 1, 1, 0, 1, 1], [1, 1, 0, 1, 0, 1, 0, 0, 1, 0], [0, 0, 1, 1, 0, 1, 1, 1, 0, 1], [1, 1, 0, 0, 1, 1, 0, 0, 1, 1], [1, 0, 0, 0, 0, 1, 0, 1, 1, 1], [0, 1, 1, 1, 0, 0, 1, 1, 0, 0], [0], [], [], [], [], [], [], []]}


# 평균 임베딩

In [71]:
import pickle
with open('emb_avg2.pickle','rb') as fw:
    movie_emb = pickle.load(fw)

In [72]:
path1 = 'word2vec_model_char/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
end = torch.ones(1,1,384)

for filename in filenames:
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'surprise ending'==keyword:
                    cult = 1
        
    embs = torch.zeros(1,len(movie_emb[movie_name])+1,384)
    for i,emb in enumerate(movie_emb[movie_name]):
        emb = torch.from_numpy(np.nan_to_num(emb))
        embs[0][i] = emb
        
    embs[0][-1] = end
        
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
        #valid_data.append([embs, [cult]])

In [73]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

963
120
121


In [74]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:08<00:00, 116.86it/s]


 Average training loss: 1.06


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 806.25it/s]

 Average valid loss: 1.04
AUC_SCORE:  0.6374223602484472  acc:  0.475 f1:  0.4424778761061947


...

 Average valid loss: 0.98
AUC_SCORE:  0.6972049689440993  acc:  0.625 f1:  0.4705882352941177
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 794.70it/s]


 Average valid loss: 0.90
AUC_SCORE:  0.7428785607196402  acc:  0.6611570247933884 f1:  0.4938271604938272


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 791.15it/s]

 Average valid loss: 0.88
AUC_SCORE:  0.7631184407796101  acc:  0.6776859504132231 f1:  0.5411764705882354
BEST MODEL
AUC_SCORE:  0.7631184407796101  acc:  0.6776859504132231 f1:  0.5411764705882354
{'auc_micro': 0.7631184407796101, 'acc': 0.6776859504132231, 'f1': 0.5411764705882354, 'pred': [[0, 0, 1, 1, 0, 1, 1, 1, 0, 1], [0, 0, 1, 0, 1, 0, 0, 0, 1, 1], [1, 1, 0, 1, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 1, 0, 0, 0, 1, 0], [1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 1, 0, 1, 1, 1], [1, 0, 0, 1, 0, 0, 0, 1, 1, 0], [1, 1, 1, 0, 0, 1, 1, 1, 0, 0], [0, 1, 1, 1, 0, 1, 0, 1, 0, 0], [0, 1, 0, 1, 1, 1, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0, 1, 1, 1, 0], [1, 1, 0, 0, 0, 0, 1, 1, 1, 1], [0], [], [], [], [], [], [], []]}


# 그냥 metapath

In [66]:
path1 = 'word2vec_model_char/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'surprise ending'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [67]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

963
120
121


In [68]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:08<00:00, 111.14it/s]


 Average training loss: 1.07


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 780.99it/s]

 Average valid loss: 1.05
AUC_SCORE:  0.5559006211180124  acc:  0.31666666666666665 f1:  0.39705882352941174


...

 Average valid loss: 0.98
AUC_SCORE:  0.7263198757763976  acc:  0.6833333333333333 f1:  0.5365853658536585
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 787.20it/s]


 Average valid loss: 0.86
AUC_SCORE:  0.7732383808095952  acc:  0.6694214876033058 f1:  0.48717948717948717


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 814.25it/s]

 Average valid loss: 0.85
AUC_SCORE:  0.7908545727136431  acc:  0.6776859504132231 f1:  0.5063291139240508
BEST MODEL
AUC_SCORE:  0.7908545727136431  acc:  0.6776859504132231 f1:  0.5063291139240508
{'auc_micro': 0.7908545727136431, 'acc': 0.6776859504132231, 'f1': 0.5063291139240508, 'pred': [[0, 0, 1, 1, 0, 1, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 1, 1], [0, 1, 0, 1, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 1, 0, 0, 1, 1, 0], [1, 0, 1, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 1, 1, 1], [1, 0, 0, 0, 1, 0, 0, 1, 1, 0], [1, 1, 1, 0, 1, 0, 1, 1, 0, 0], [0, 1, 1, 1, 1, 1, 1, 1, 0, 0], [0, 0, 0, 1, 1, 1, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0, 1, 1, 0, 0], [0, 1, 0, 0, 0, 0, 1, 1, 0, 0], [0], [], [], [], [], [], [], []]}


# Main_act

In [69]:
path1 = 'word2vec_model_extra+emb/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'surprise ending'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [70]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:08<00:00, 114.71it/s]


 Average training loss: 1.05


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 828.20it/s]

 Average valid loss: 1.04
AUC_SCORE:  0.5974378881987578  acc:  0.38333333333333336 f1:  0.3934426229508196


...

 Average valid loss: 0.94
AUC_SCORE:  0.7302018633540374  acc:  0.6583333333333333 f1:  0.48101265822784806
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 788.58it/s]


 Average valid loss: 0.82
AUC_SCORE:  0.8050974512743628  acc:  0.7272727272727273 f1:  0.5599999999999999


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 785.08it/s]

 Average valid loss: 0.80
AUC_SCORE:  0.81784107946027  acc:  0.71900826446281 f1:  0.5526315789473684
BEST MODEL
AUC_SCORE:  0.81784107946027  acc:  0.71900826446281 f1:  0.5526315789473684
{'auc_micro': 0.81784107946027, 'acc': 0.71900826446281, 'f1': 0.5526315789473684, 'pred': [[0, 0, 1, 1, 0, 1, 1, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 0, 0, 1, 1, 0], [1, 0, 1, 0, 0, 0, 1, 1, 0, 0], [0, 0, 1, 0, 0, 1, 0, 1, 1, 1], [1, 0, 0, 1, 0, 0, 0, 1, 0, 1], [1, 1, 0, 0, 1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 0, 1, 0, 1, 0, 0], [0, 1, 0, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 1, 0, 0], [0, 1, 0, 0, 0, 1, 1, 1, 0, 0], [0], [], [], [], [], [], [], []]}


# pos

In [53]:
path1 = 'word2vec_model_extra+pos/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,400)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'surprise ending'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,400)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [56]:
bert2 = BertModel(d_model=400).to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=25)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 962/962 [00:08<00:00, 113.24it/s]


 Average training loss: 1.06


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 807.40it/s]


 Average valid loss: 1.05
AUC_SCORE:  0.5368788819875777  acc:  0.45 f1:  0.3773584905660377
Test


...

 Average valid loss: 0.99
AUC_SCORE:  0.6777950310559006  acc:  0.625 f1:  0.4
Test


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 796.67it/s]


 Average valid loss: 0.91
AUC_SCORE:  0.7389162561576355  acc:  0.6666666666666666 f1:  0.47368421052631576


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 808.34it/s]

 Average valid loss: 0.92
AUC_SCORE:  0.7430845017051914  acc:  0.6583333333333333 f1:  0.5060240963855421
BEST MODEL
AUC_SCORE:  0.7430845017051914  acc:  0.6583333333333333 f1:  0.5060240963855421
{'auc_micro': 0.7430845017051914, 'acc': 0.6583333333333333, 'f1': 0.5060240963855421, 'pred': [[0, 0, 1, 0, 1, 1, 1, 0, 0, 0], [0, 1, 1, 1, 0, 1, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0, 1, 1, 0, 1], [1, 0, 0, 1, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 1, 1, 0, 1, 0], [0, 1, 0, 0, 1, 0, 1, 1, 1, 1], [0, 0, 1, 1, 0, 0, 1, 0, 0, 1], [1, 1, 0, 0, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 0, 1, 0, 0, 1], [0, 0, 1, 1, 1, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 1, 1, 0, 0, 1], [1, 0, 0, 0, 1, 1, 0, 0, 1, 0], [], [], [], [], [], [], [], []]}


# text_rank

In [88]:
path1 = 'word2vec_model_text/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'surprise ending'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [89]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 960/960 [00:08<00:00, 118.56it/s]


 Average training loss: 1.08


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 820.88it/s]


 Average valid loss: 1.05
AUC_SCORE:  0.5896739130434783  acc:  0.2833333333333333 f1:  0.3857142857142858
Test


...

 Average valid loss: 1.00
AUC_SCORE:  0.7123447204968943  acc:  0.6583333333333333 f1:  0.5060240963855421
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 860.02it/s]


 Average valid loss: 0.83
AUC_SCORE:  0.7991004497751124  acc:  0.743801652892562 f1:  0.5866666666666667


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 859.62it/s]

 Average valid loss: 0.85
AUC_SCORE:  0.7976011994002998  acc:  0.7024793388429752 f1:  0.55
BEST MODEL
AUC_SCORE:  0.7976011994002998  acc:  0.7024793388429752 f1:  0.55
{'auc_micro': 0.7976011994002998, 'acc': 0.7024793388429752, 'f1': 0.55, 'pred': [[0, 0, 1, 1, 0, 1, 0, 1, 0, 0], [0, 0, 1, 1, 1, 0, 0, 0, 1, 1], [0, 1, 0, 0, 0, 0, 0, 0, 1, 0], [1, 1, 0, 0, 1, 0, 0, 1, 0, 1], [1, 0, 1, 0, 0, 0, 1, 1, 0, 0], [0, 0, 1, 0, 0, 1, 0, 1, 1, 1], [1, 0, 0, 1, 1, 0, 0, 1, 0, 1], [1, 1, 1, 0, 1, 0, 1, 1, 0, 0], [0, 1, 1, 1, 1, 1, 0, 1, 0, 0], [1, 0, 0, 0, 1, 1, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 0, 1, 0, 0], [0], [], [], [], [], [], [], []]}


In [90]:
path1 = 'word2vec_model_text2/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'surprise ending'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [91]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 960/960 [00:08<00:00, 117.91it/s]


 Average training loss: 1.07


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 831.97it/s]

 Average valid loss: 1.05
AUC_SCORE:  0.5993788819875775  acc:  0.275 f1:  0.36496350364963503


...

 Average valid loss: 0.95
AUC_SCORE:  0.7309782608695652  acc:  0.6833333333333333 f1:  0.5250000000000001
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 857.88it/s]


 Average valid loss: 0.89
AUC_SCORE:  0.7582458770614693  acc:  0.6776859504132231 f1:  0.5185185185185185


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 853.66it/s]

 Average valid loss: 0.87
AUC_SCORE:  0.7616191904047976  acc:  0.6611570247933884 f1:  0.5060240963855421
BEST MODEL
AUC_SCORE:  0.7616191904047976  acc:  0.6611570247933884 f1:  0.5060240963855421
{'auc_micro': 0.7616191904047976, 'acc': 0.6611570247933884, 'f1': 0.5060240963855421, 'pred': [[0, 0, 1, 0, 0, 1, 0, 1, 0, 0], [0, 0, 1, 1, 1, 0, 0, 0, 0, 1], [1, 1, 0, 1, 0, 0, 0, 1, 1, 0], [1, 1, 0, 0, 1, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 1, 1, 0, 1], [1, 0, 0, 0, 0, 1, 0, 1, 1, 1], [0, 0, 0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 0, 1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 1, 1, 1, 0, 0], [0, 0, 0, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 1, 0, 0], [0, 1, 0, 0, 0, 1, 1, 1, 0, 0], [0], [], [], [], [], [], [], []]}


In [92]:
path1 = 'word2vec_model_text3/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'surprise ending'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [93]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 960/960 [00:08<00:00, 117.94it/s]


 Average training loss: 1.06


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 839.52it/s]

 Average valid loss: 1.06
AUC_SCORE:  0.4553571428571429  acc:  0.31666666666666665 f1:  0.3492063492063492


...

 Average valid loss: 1.06
AUC_SCORE:  0.6875000000000001  acc:  0.6833333333333333 f1:  0.4722222222222222
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 861.14it/s]


 Average valid loss: 0.90
AUC_SCORE:  0.7721139430284858  acc:  0.7272727272727273 f1:  0.5352112676056338


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 856.57it/s]

 Average valid loss: 0.89
AUC_SCORE:  0.7728635682158921  acc:  0.6611570247933884 f1:  0.4675324675324676
BEST MODEL
AUC_SCORE:  0.7728635682158921  acc:  0.6611570247933884 f1:  0.4675324675324676
{'auc_micro': 0.7728635682158921, 'acc': 0.6611570247933884, 'f1': 0.4675324675324676, 'pred': [[0, 0, 1, 0, 0, 1, 0, 1, 0, 0], [0, 0, 1, 1, 0, 0, 1, 1, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 1, 0], [1, 1, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 1, 1, 0, 1], [0, 0, 0, 0, 0, 1, 0, 1, 1, 1], [1, 0, 0, 0, 1, 0, 0, 1, 0, 1], [1, 1, 0, 0, 0, 0, 1, 1, 0, 0], [0, 1, 1, 1, 1, 1, 0, 1, 0, 0], [1, 0, 0, 1, 1, 1, 0, 0, 0, 1], [0, 0, 1, 0, 1, 0, 1, 1, 1, 0], [1, 1, 0, 0, 0, 0, 0, 1, 0, 0], [0], [], [], [], [], [], [], []]}


# 거리로 분석

In [156]:
path1 = 'word2vec_model_dim=384/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'surprise ending'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1

    ans = torch.zeros(1,1,512)
    for i in range(scene_num):
        if i ==0:
            continue
        #ans[0,0,i] += util.cos_sim(embs[0][i],embs[0][i-1])[0][0]
        ans[0,0,i] += torch.cdist(embs[0][i].unsqueeze(0),embs[0][i-1].unsqueeze(0))[0][0]
        
    embs = ans
    
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [157]:
bert2 = BertModel(d_model=512).to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=20)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 929/929 [00:07<00:00, 127.24it/s]


 Average training loss: 1.07


100%|████████████████████████████████████████| 117/117 [00:00<00:00, 980.24it/s]


 Average valid loss: 1.06
AUC_SCORE:  0.5272873194221509  acc:  0.4700854700854701 f1:  0.3404255319148936
EPOCHS:2
TRAIN


...

 Average training loss: 1.02


100%|████████████████████████████████████████| 117/117 [00:00<00:00, 989.84it/s]


 Average valid loss: 1.13
AUC_SCORE:  0.4562600321027287  acc:  0.47863247863247865 f1:  0.2823529411764706


100%|████████████████████████████████████████| 117/117 [00:00<00:00, 985.66it/s]

 Average valid loss: 1.06
AUC_SCORE:  0.5497592295345104  acc:  0.41025641025641024 f1:  0.3551401869158879
BEST MODEL
AUC_SCORE:  0.5497592295345104  acc:  0.41025641025641024 f1:  0.3551401869158879
{'auc_micro': 0.5497592295345104, 'acc': 0.41025641025641024, 'f1': 0.3551401869158879, 'pred': [[1, 1, 1, 1, 1, 1, 1, 1, 0, 1], [1, 1, 0, 1, 1, 0, 0, 0, 1, 1], [0, 1, 1, 0, 1, 1, 1, 0, 1, 0], [1, 1, 1, 1, 1, 1, 0, 0, 1, 0], [0, 1, 0, 1, 1, 1, 1, 1, 1, 1], [1, 1, 0, 1, 0, 1, 1, 1, 0, 1], [1, 1, 1, 0, 1, 1, 0, 0, 1, 1], [0, 1, 1, 1, 1, 1, 0, 1, 1, 0], [1, 0, 1, 0, 1, 1, 0, 1, 0, 1], [1, 1, 1, 0, 1, 0, 0, 0, 0, 1], [1, 1, 1, 1, 0, 0, 1, 1, 0, 1], [1, 0, 1, 0, 1, 1, 0], [], [], [], [], [], [], []]}
